In [3]:
import sys
# sys.path.append(r'E:/0code')
sys.path.append(r'/home/wyf/0code')
# sys.path.append(r'/home/wangyf226/0code')
# sys.path.append(r'/BIGDATA1/nsccgz_yfdu_1/asc19/wyf/pyml')

In [15]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime
import gensim
import re
import smart_open
from pyml.feature_extraction.text import CountVectorizer
from pyml.linear_model.classification import LogisticClassifier
from pyml.neighbors.classification import KNeighborsClassifier
from pyml.metrics.classification import precision_score
from pyml.model_selection import KFold
from pyml.model_selection import ShuffleSplit
from pyml.preprocessing import StandardScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 读取数据集

1. 读取训练数据和测试数据为字符串的列表
2. 读取训练集label，并转换为数字格式

In [5]:
def read_train_text_to_list(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        contents = f.readlines()
    lines = [l.strip() for l in contents]
    return lines

In [6]:
# train_ori_X = read_train_text_to_list('../data/trainData.txt')
train_ori_Y = read_train_text_to_list('../data/trainLabel.txt')
train_ori_Y = np.array([int(y) for y in train_ori_Y])
# test_ori_X = read_train_text_to_list('../data/testData.txt')

In [7]:
def read_raw_documents(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="UTF-8") as f:
        for i, line in enumerate(f):
            if tokens_only:
                # 变小写，去标点符号，分词
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [8]:
train_sentences = list(read_raw_documents('../data/trainData.txt'))
test_sentences = list(read_raw_documents('../data/testData.txt', tokens_only=True))

# 数据预处理 & 特征工程
1. Count Vectors as feature
2. TF-IDF Vectors as festures
3. Word Embeddings as features
4. Text/NLP based features
5. Topic Models as features

In [9]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [10]:
model.build_vocab(train_sentences)

In [11]:
model.train(train_sentences, total_examples=model.corpus_count, epochs=model.epochs)

In [12]:
n_train_samples = len(train_sentences)
n_test_samples = len(test_sentences)
vector_size = 50
train_X = np.zeros((n_train_samples, vector_size))
test_X = np.zeros((n_test_samples, vector_size))
for i in range(0, n_train_samples):
    train_X[i] = model.infer_vector(train_sentences[i][0])
for i in range(0, n_test_samples):
    test_X[i] = model.infer_vector(test_sentences[i])

In [13]:
train_Y = train_ori_Y

In [14]:
train_Y.shape

(24000,)

# 交叉验证-寻找最好的参数

In [16]:
num_parameter = 2
learning_rate_range = [0.15, 0.05]
num_iterations = [30000, 50000]
n_splits = 2
ms = ShuffleSplit(n_splits=n_splits)
scores = np.zeros((len(learning_rate_range)))
for train_indices, test_indices in ms.split(train_X):
    for i in range(0, num_parameter):
        clf = LogisticClassifier(learning_rate=learning_rate_range[i], num_iterations=num_iterations[i])
        clf.fit(train_X[train_indices], train_Y[train_indices], watch=True)
        y_pred = clf.predict(train_X[test_indices])
        score = precision_score(train_Y[test_indices], y_pred)
        print('i : {} score: {}'.format(i, score))
        scores[i] += score
    print(scores)
avg_scores = scores/n_splits
print(avg_scores)
print("best k ", np.argmax(avg_scores))

Cost after iteration 0: 2.267442
Cost after iteration 100: 0.745192
Cost after iteration 200: 0.536773
Cost after iteration 300: 0.473392
Cost after iteration 400: 0.447260
Cost after iteration 500: 0.434830
Cost after iteration 600: 0.428433
Cost after iteration 700: 0.424990
Cost after iteration 800: 0.423083
Cost after iteration 900: 0.422001
Cost after iteration 1000: 0.421371
Cost after iteration 1100: 0.420994
Cost after iteration 1200: 0.420760
Cost after iteration 1300: 0.420608
Cost after iteration 1400: 0.420505
Cost after iteration 1500: 0.420430
Cost after iteration 1600: 0.420373
Cost after iteration 1700: 0.420327
Cost after iteration 1800: 0.420289
Cost after iteration 1900: 0.420255
Cost after iteration 2000: 0.420226
Cost after iteration 2100: 0.420199
Cost after iteration 2200: 0.420174
Cost after iteration 2300: 0.420151
Cost after iteration 2400: 0.420130
Cost after iteration 2500: 0.420110
Cost after iteration 2600: 0.420091
Cost after iteration 2700: 0.420073
Cost

Cost after iteration 22500: 0.419780
Cost after iteration 22600: 0.419780
Cost after iteration 22700: 0.419780
Cost after iteration 22800: 0.419780
Cost after iteration 22900: 0.419780
Cost after iteration 23000: 0.419780
Cost after iteration 23100: 0.419780
Cost after iteration 23200: 0.419780
Cost after iteration 23300: 0.419780
Cost after iteration 23400: 0.419780
Cost after iteration 23500: 0.419780
Cost after iteration 23600: 0.419780
Cost after iteration 23700: 0.419780
Cost after iteration 23800: 0.419780
Cost after iteration 23900: 0.419780
Cost after iteration 24000: 0.419780
Cost after iteration 24100: 0.419780
Cost after iteration 24200: 0.419780
Cost after iteration 24300: 0.419780
Cost after iteration 24400: 0.419780
Cost after iteration 24500: 0.419780
Cost after iteration 24600: 0.419780
Cost after iteration 24700: 0.419780
Cost after iteration 24800: 0.419780
Cost after iteration 24900: 0.419780
Cost after iteration 25000: 0.419780
Cost after iteration 25100: 0.419780
C

Cost after iteration 14900: 0.419828
Cost after iteration 15000: 0.419827
Cost after iteration 15100: 0.419826
Cost after iteration 15200: 0.419825
Cost after iteration 15300: 0.419824
Cost after iteration 15400: 0.419823
Cost after iteration 15500: 0.419823
Cost after iteration 15600: 0.419822
Cost after iteration 15700: 0.419821
Cost after iteration 15800: 0.419820
Cost after iteration 15900: 0.419819
Cost after iteration 16000: 0.419819
Cost after iteration 16100: 0.419818
Cost after iteration 16200: 0.419817
Cost after iteration 16300: 0.419817
Cost after iteration 16400: 0.419816
Cost after iteration 16500: 0.419815
Cost after iteration 16600: 0.419814
Cost after iteration 16700: 0.419814
Cost after iteration 16800: 0.419813
Cost after iteration 16900: 0.419813
Cost after iteration 17000: 0.419812
Cost after iteration 17100: 0.419811
Cost after iteration 17200: 0.419811
Cost after iteration 17300: 0.419810
Cost after iteration 17400: 0.419810
Cost after iteration 17500: 0.419809
C

Cost after iteration 37200: 0.419781
Cost after iteration 37300: 0.419781
Cost after iteration 37400: 0.419781
Cost after iteration 37500: 0.419781
Cost after iteration 37600: 0.419781
Cost after iteration 37700: 0.419781
Cost after iteration 37800: 0.419781
Cost after iteration 37900: 0.419781
Cost after iteration 38000: 0.419781
Cost after iteration 38100: 0.419781
Cost after iteration 38200: 0.419781
Cost after iteration 38300: 0.419781
Cost after iteration 38400: 0.419781
Cost after iteration 38500: 0.419781
Cost after iteration 38600: 0.419781
Cost after iteration 38700: 0.419781
Cost after iteration 38800: 0.419781
Cost after iteration 38900: 0.419781
Cost after iteration 39000: 0.419781
Cost after iteration 39100: 0.419781
Cost after iteration 39200: 0.419781
Cost after iteration 39300: 0.419781
Cost after iteration 39400: 0.419781
Cost after iteration 39500: 0.419781
Cost after iteration 39600: 0.419781
Cost after iteration 39700: 0.419781
Cost after iteration 39800: 0.419781
C

Cost after iteration 9600: 0.421612
Cost after iteration 9700: 0.421612
Cost after iteration 9800: 0.421612
Cost after iteration 9900: 0.421612
Cost after iteration 10000: 0.421612
Cost after iteration 10100: 0.421612
Cost after iteration 10200: 0.421612
Cost after iteration 10300: 0.421612
Cost after iteration 10400: 0.421612
Cost after iteration 10500: 0.421612
Cost after iteration 10600: 0.421612
Cost after iteration 10700: 0.421612
Cost after iteration 10800: 0.421612
Cost after iteration 10900: 0.421612
Cost after iteration 11000: 0.421612
Cost after iteration 11100: 0.421612
Cost after iteration 11200: 0.421612
Cost after iteration 11300: 0.421612
Cost after iteration 11400: 0.421612
Cost after iteration 11500: 0.421612
Cost after iteration 11600: 0.421612
Cost after iteration 11700: 0.421612
Cost after iteration 11800: 0.421612
Cost after iteration 11900: 0.421612
Cost after iteration 12000: 0.421612
Cost after iteration 12100: 0.421612
Cost after iteration 12200: 0.421612
Cost 

Cost after iteration 1900: 0.432439
Cost after iteration 2000: 0.431096
Cost after iteration 2100: 0.429987
Cost after iteration 2200: 0.429067
Cost after iteration 2300: 0.428302
Cost after iteration 2400: 0.427662
Cost after iteration 2500: 0.427124
Cost after iteration 2600: 0.426671
Cost after iteration 2700: 0.426287
Cost after iteration 2800: 0.425960
Cost after iteration 2900: 0.425680
Cost after iteration 3000: 0.425440
Cost after iteration 3100: 0.425231
Cost after iteration 3200: 0.425050
Cost after iteration 3300: 0.424891
Cost after iteration 3400: 0.424750
Cost after iteration 3500: 0.424626
Cost after iteration 3600: 0.424514
Cost after iteration 3700: 0.424413
Cost after iteration 3800: 0.424322
Cost after iteration 3900: 0.424239
Cost after iteration 4000: 0.424162
Cost after iteration 4100: 0.424091
Cost after iteration 4200: 0.424025
Cost after iteration 4300: 0.423963
Cost after iteration 4400: 0.423905
Cost after iteration 4500: 0.423850
Cost after iteration 4600: 0

Cost after iteration 24300: 0.421657
Cost after iteration 24400: 0.421656
Cost after iteration 24500: 0.421655
Cost after iteration 24600: 0.421654
Cost after iteration 24700: 0.421654
Cost after iteration 24800: 0.421653
Cost after iteration 24900: 0.421652
Cost after iteration 25000: 0.421651
Cost after iteration 25100: 0.421650
Cost after iteration 25200: 0.421650
Cost after iteration 25300: 0.421649
Cost after iteration 25400: 0.421648
Cost after iteration 25500: 0.421647
Cost after iteration 25600: 0.421647
Cost after iteration 25700: 0.421646
Cost after iteration 25800: 0.421645
Cost after iteration 25900: 0.421645
Cost after iteration 26000: 0.421644
Cost after iteration 26100: 0.421644
Cost after iteration 26200: 0.421643
Cost after iteration 26300: 0.421642
Cost after iteration 26400: 0.421642
Cost after iteration 26500: 0.421641
Cost after iteration 26600: 0.421641
Cost after iteration 26700: 0.421640
Cost after iteration 26800: 0.421639
Cost after iteration 26900: 0.421639
C

Cost after iteration 46600: 0.421612
Cost after iteration 46700: 0.421612
Cost after iteration 46800: 0.421612
Cost after iteration 46900: 0.421612
Cost after iteration 47000: 0.421612
Cost after iteration 47100: 0.421612
Cost after iteration 47200: 0.421612
Cost after iteration 47300: 0.421612
Cost after iteration 47400: 0.421612
Cost after iteration 47500: 0.421612
Cost after iteration 47600: 0.421612
Cost after iteration 47700: 0.421612
Cost after iteration 47800: 0.421612
Cost after iteration 47900: 0.421612
Cost after iteration 48000: 0.421612
Cost after iteration 48100: 0.421612
Cost after iteration 48200: 0.421612
Cost after iteration 48300: 0.421612
Cost after iteration 48400: 0.421612
Cost after iteration 48500: 0.421612
Cost after iteration 48600: 0.421612
Cost after iteration 48700: 0.421612
Cost after iteration 48800: 0.421612
Cost after iteration 48900: 0.421612
Cost after iteration 49000: 0.421612
Cost after iteration 49100: 0.421612
Cost after iteration 49200: 0.421612
C

# 训练模型

In [17]:
clf = LogisticClassifier(learning_rate=0.1, num_iterations=50000)
clf.fit(train_X, train_Y, watch=True)

Cost after iteration 0: 1.922223
Cost after iteration 100: 0.983062
Cost after iteration 200: 0.696180
Cost after iteration 300: 0.575996
Cost after iteration 400: 0.514294
Cost after iteration 500: 0.479213
Cost after iteration 600: 0.458113
Cost after iteration 700: 0.444977
Cost after iteration 800: 0.436613
Cost after iteration 900: 0.431205
Cost after iteration 1000: 0.427670
Cost after iteration 1100: 0.425340
Cost after iteration 1200: 0.423795
Cost after iteration 1300: 0.422764
Cost after iteration 1400: 0.422073
Cost after iteration 1500: 0.421608
Cost after iteration 1600: 0.421293
Cost after iteration 1700: 0.421080
Cost after iteration 1800: 0.420934
Cost after iteration 1900: 0.420835
Cost after iteration 2000: 0.420766
Cost after iteration 2100: 0.420719
Cost after iteration 2200: 0.420686
Cost after iteration 2300: 0.420664
Cost after iteration 2400: 0.420648
Cost after iteration 2500: 0.420637
Cost after iteration 2600: 0.420629
Cost after iteration 2700: 0.420623
Cost

Cost after iteration 22500: 0.420603
Cost after iteration 22600: 0.420603
Cost after iteration 22700: 0.420603
Cost after iteration 22800: 0.420603
Cost after iteration 22900: 0.420603
Cost after iteration 23000: 0.420603
Cost after iteration 23100: 0.420603
Cost after iteration 23200: 0.420603
Cost after iteration 23300: 0.420603
Cost after iteration 23400: 0.420603
Cost after iteration 23500: 0.420603
Cost after iteration 23600: 0.420603
Cost after iteration 23700: 0.420603
Cost after iteration 23800: 0.420603
Cost after iteration 23900: 0.420603
Cost after iteration 24000: 0.420603
Cost after iteration 24100: 0.420603
Cost after iteration 24200: 0.420603
Cost after iteration 24300: 0.420603
Cost after iteration 24400: 0.420603
Cost after iteration 24500: 0.420603
Cost after iteration 24600: 0.420603
Cost after iteration 24700: 0.420603
Cost after iteration 24800: 0.420603
Cost after iteration 24900: 0.420603
Cost after iteration 25000: 0.420603
Cost after iteration 25100: 0.420603
C

Cost after iteration 44700: 0.420603
Cost after iteration 44800: 0.420603
Cost after iteration 44900: 0.420603
Cost after iteration 45000: 0.420603
Cost after iteration 45100: 0.420603
Cost after iteration 45200: 0.420603
Cost after iteration 45300: 0.420603
Cost after iteration 45400: 0.420603
Cost after iteration 45500: 0.420603
Cost after iteration 45600: 0.420603
Cost after iteration 45700: 0.420603
Cost after iteration 45800: 0.420603
Cost after iteration 45900: 0.420603
Cost after iteration 46000: 0.420603
Cost after iteration 46100: 0.420603
Cost after iteration 46200: 0.420603
Cost after iteration 46300: 0.420603
Cost after iteration 46400: 0.420603
Cost after iteration 46500: 0.420603
Cost after iteration 46600: 0.420603
Cost after iteration 46700: 0.420603
Cost after iteration 46800: 0.420603
Cost after iteration 46900: 0.420603
Cost after iteration 47000: 0.420603
Cost after iteration 47100: 0.420603
Cost after iteration 47200: 0.420603
Cost after iteration 47300: 0.420603
C

# 训练模型写入结果

In [18]:
y_pred = clf.predict(test_X)
sub = pd.DataFrame(y_pred)
sub.to_csv('../results/'+'doc2vec-lr-'+ str(datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")) + ".csv", index=0, header=None, index_label=None)